In [12]:
import os
import shutil
import random

original_image_folder = "/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/totalImages"
original_sov_label_folder = "/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelSOV"
original_aorta_label_folder = "/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelAscendingAorta"

# New folders to hold the split dataset
train_image_folder = "./train_images"
test_image_folder  = "./test_images"

train_sov_label_folder = "./train_labelSOV"
test_sov_label_folder  = "./test_labelSOV"

train_aorta_label_folder = "./train_labelAorta"
test_aorta_label_folder  = "./test_labelAorta"

# Create directories if they don't exist
os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(test_image_folder, exist_ok=True)

os.makedirs(train_sov_label_folder, exist_ok=True)
os.makedirs(test_sov_label_folder, exist_ok=True)

os.makedirs(train_aorta_label_folder, exist_ok=True)
os.makedirs(test_aorta_label_folder, exist_ok=True)

# Gather all images in the original folder
all_images = [
    f for f in os.listdir(original_image_folder)
    if f.lower().endswith((".jpg", ".png", ".jpeg"))
]

# 80% for train, 20% for test
split_idx = int(0.8 * len(all_images))
train_files = all_images[:split_idx]
test_files  = all_images[split_idx:]

def copy_label_if_exists(label_folder_src, label_folder_dst, base_name):
    """
    Copies the label file if it exists for the given image base name.
    E.g. label file is base_name + '.txt'.
    """
    label_file = base_name + ".txt"
    src_path = os.path.join(label_folder_src, label_file)
    if os.path.exists(src_path):
        dst_path = os.path.join(label_folder_dst, label_file)
        shutil.copy(src_path, dst_path)

# Copy training images and labels
for img_file in train_files:
    # Copy the image
    src_img = os.path.join(original_image_folder, img_file)
    dst_img = os.path.join(train_image_folder, img_file)
    shutil.copy(src_img, dst_img)

    # Also copy the SOV and Aorta label
    base_name, _ = os.path.splitext(img_file)
    copy_label_if_exists(original_sov_label_folder, train_sov_label_folder, base_name)
    copy_label_if_exists(original_aorta_label_folder, train_aorta_label_folder, base_name)

# Copy testing images and labels
for img_file in test_files:
    # Copy the image
    src_img = os.path.join(original_image_folder, img_file)
    dst_img = os.path.join(test_image_folder, img_file)
    shutil.copy(src_img, dst_img)

    # Also copy the SOV and Aorta label
    base_name, _ = os.path.splitext(img_file)
    copy_label_if_exists(original_sov_label_folder, test_sov_label_folder, base_name)
    copy_label_if_exists(original_aorta_label_folder, test_aorta_label_folder, base_name)

print("Train/Test split complete.")
print(f"Train images: {len(train_files)}")
print(f"Test images: {len(test_files)}")


Train/Test split complete.
Train images: 967
Test images: 242


In [ ]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from ultralytics import YOLO
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
from PIL import Image
import cv2
import torch.nn as nn

##############################################
# Define paths (adjust these as needed)
##############################################
train_image_folder = "./train_images"  # must be defined previously
test_image_folder  = "./test_images"   # must be defined previously
feature_save_train = "extracted_features_total"
feature_save_test  = "extracted_features_total"

##############################################
# Feature Extraction Code (using global average pooling)
##############################################
# This code assumes that the YOLO neck returns a tensor of shape [B, 32, 160, 160].
# Global average pooling will convert it to shape [B, 32].
yolo = YOLO("yolov8n.pt")  # or your custom .pt
yolo.model.eval()
backbone = yolo.model.model[0]
neck = yolo.model.model[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
backbone.to(device)
neck.to(device)
backbone.eval()
neck.eval()

class MyImagesDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_files = [f for f in os.listdir(image_folder)
                            if f.lower().endswith((".jpg", ".png", ".jpeg"))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_name

transform = T.Compose([
    T.Resize((640, 640)),
    T.ToTensor()
])

# Create datasets and loaders for feature extraction (batch_size=1 for simplicity)
train_dataset_img = MyImagesDataset(train_image_folder, transform=transform)
test_dataset_img  = MyImagesDataset(test_image_folder, transform=transform)
train_loader_img = DataLoader(train_dataset_img, batch_size=1, shuffle=False)
test_loader_img  = DataLoader(test_dataset_img, batch_size=1, shuffle=False)

def extract_neck_features(dataloader, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    with torch.no_grad():
        for images, img_names in dataloader:
            images = images.to(device)
            # Run images through YOLO backbone and neck
            backbone_outputs = backbone(images)
            neck_outputs = neck(backbone_outputs)
            if isinstance(neck_outputs, list):
                feats = neck_outputs[0]  # Expected shape: [B, 32, 160, 160]
            else:
                feats = neck_outputs
            # Apply global average pooling: from [B, 32, 160, 160] to [B, 32]
            feats = F.adaptive_avg_pool2d(feats, (4, 4))
            feats = feats.view(feats.size(0), -1)
            feats = feats.cpu().numpy()
            for i, name in enumerate(img_names):
                base_name, _ = os.path.splitext(name)
                out_path = os.path.join(save_dir, base_name + ".npy")
                np.save(out_path, feats[i])
    print(f"Done! Extracted neck features saved in '{save_dir}'.")

# Run feature extraction for train and test images
extract_neck_features(train_loader_img, feature_save_train)
extract_neck_features(test_loader_img, feature_save_test)


# print(count)
import numpy as np 
test=np.load("/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/extracted_features_total/P G - 3mensio Screen Recording_longitudinal_view_242_brightness_contrast.npy")
print(test)

Done! Extracted neck features saved in 'extracted_features_total'.
Done! Extracted neck features saved in 'extracted_features_total'.
[      1.282     0.67313      0.7362     0.75269     0.62895     0.68957     0.80632     0.85958     0.60634      1.2078     0.80859      1.0512     0.51512     0.66284     0.72925      1.2712      1.6092      1.2039      1.1945      1.1908      1.0989      1.2441      1.3387      1.4315     0.79884      1.2955
      1.3847      1.2603      1.0451      1.2707      1.2418      2.2747      2.2949      2.5207      2.6004      2.2886      2.4708      2.4347      2.5919      2.4617      2.0269      2.2646      2.5667      2.4806      2.1314      2.3822      2.6582      2.5102      1.3888     0.50115     0.45272     0.49158
     0.44819     0.54675     0.57034     0.63219     0.35534     0.55983     0.58893     0.60479     0.48392     0.59202     0.51163      1.3459     0.87551     0.15504     0.11292     0.10387     0.15212     0.15387     0.14129     0.13978

In [14]:
import numpy as np
t=np.load("/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/extracted_features_test/P G - 3mensio Screen Recording_longitudinal_view_242_brightness_contrast.npy")
print(t.shape)

(512,)


In [15]:

# ##############################################
# # Now define the dataset for bounding-box regression
# ##############################################
# class BBFeaturesDataset(Dataset):
#     def __init__(self, feature_dir, sov_label_dir, aorta_label_dir, expected_feat_dim):
#         self.feature_dir = feature_dir
#         self.sov_label_dir = sov_label_dir
#         self.aorta_label_dir = aorta_label_dir
#         self.expected_feat_dim = expected_feat_dim  # expected_feat_dim should be 32 + 4 = 36

#         self.feature_files = [f for f in os.listdir(feature_dir) if f.endswith(".npy")]

#     def __len__(self):
#         return len(self.feature_files)

#     def __getitem__(self, idx):
#         feature_file = self.feature_files[idx]
#         base_name = feature_file.replace(".npy", "")
        
#         # 1) Load YOLO feature (expected shape [32])
#         feat_path = os.path.join(self.feature_dir, feature_file)
#         try:
#             features = np.load(feat_path)
#         except Exception:
#             return None
#         features = torch.from_numpy(features).float()
        
#         # 2) Load SOV bounding box (assumes format: class x y w h)
#         sov_txt = os.path.join(self.sov_label_dir, base_name + ".txt")
#         try:
#             with open(sov_txt, "r") as f:
#                 line = f.readline().strip()
#                 vals = list(map(float, line.split()))
#                 sov_bbox = vals[1:]  # skip class
#         except (ValueError, FileNotFoundError):
#             return None
#         sov_bbox = torch.tensor(sov_bbox, dtype=torch.float32)
        
#         # 3) Combine features and SOV bbox -> Expected shape: [32 + 4] = [36]
#         combined_input = torch.cat([features, sov_bbox], dim=0)
#         if combined_input.shape[0] != self.expected_feat_dim:
#             return None
        
#         # 4) Load Aorta bounding box (assumes format: class x y w h)
#         aorta_txt = os.path.join(self.aorta_label_dir, base_name + ".txt")
#         try:
#             with open(aorta_txt, "r") as f:
#                 line = f.readline().strip()
#                 vals = list(map(float, line.split()))
#                 aorta_bbox = vals[1:]  # skip class
#         except (ValueError, FileNotFoundError):
#             return None
#         aorta_bbox = torch.tensor(aorta_bbox, dtype=torch.float32)
        
#         return combined_input, aorta_bbox

# def skip_invalid_collate_fn(batch):
#     valid_batch = [x for x in batch if x is not None]
#     if len(valid_batch) == 0:
#         return None
#     from torch.utils.data.dataloader import default_collate
#     return default_collate(valid_batch)

# ##############################################
# # Create the BBFeaturesDataset and split into train/test
# ##############################################
# feature_dir = feature_save_test  # or feature_save_train, depending on which split you want to test
# sov_label_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelSOV"
# aorta_label_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelAscendingAorta"

# # expected_feat_dim: 32 from pooled features + 4 for the SOV bbox = 36
# expected_feat_dim = 32 + 4

# full_bb_dataset = BBFeaturesDataset(
#     feature_dir=feature_dir,
#     sov_label_dir=sov_label_dir,
#     aorta_label_dir=aorta_label_dir,
#     expected_feat_dim=expected_feat_dim
# )

# # Split into train/test
# train_size = int(0.8 * len(full_bb_dataset))
# test_size = len(full_bb_dataset) - train_size
# train_dataset, test_dataset = random_split(full_bb_dataset, [train_size, test_size],
#                                           generator=torch.Generator().manual_seed(42))

# # Create DataLoaders
# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=skip_invalid_collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=skip_invalid_collate_fn)
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split

##############################################
# Dataset for bounding-box regression with debug prints
##############################################
class BBFeaturesDataset(Dataset):
    def __init__(self, feature_dir, sov_label_dir, aorta_label_dir, expected_feat_dim):
        self.feature_dir = feature_dir
        self.sov_label_dir = sov_label_dir
        self.aorta_label_dir = aorta_label_dir
        self.expected_feat_dim = expected_feat_dim  # expected_feat_dim should be 32 + 4 = 36

        self.feature_files = [f for f in os.listdir(feature_dir) if f.endswith(".npy")]
        # print(f"[DEBUG] Found {len(self.feature_files)} feature files in {feature_dir}")

    def __len__(self):
        return len(self.feature_files)

    # Modify __getitem__ to handle invalid SOV labels
    def __getitem__(self, idx):
        feature_file = self.feature_files[idx]
        base_name = feature_file.replace(".npy", "")

        # 1) Load YOLO feature (expected shape [32])
        feat_path = os.path.join(self.feature_dir, feature_file)
        try:
            features = np.load(feat_path)
        except Exception as e:
            # print(f"[DEBUG] Error loading features for {base_name}: {e}")
            return None
        features = torch.from_numpy(features).float()
        # print(f"[DEBUG] features shape for {base_name}: {features.shape}")

        # 2) Load SOV bounding box
        sov_txt = os.path.join(self.sov_label_dir, base_name + ".txt")
        try:
            with open(sov_txt, "r") as f:
                line = f.readline().strip()
                # Ensure all parts are numeric
                if not all(part.replace('.', '', 1).isdigit() for part in line.split()):
                    # print(f"[DEBUG] Invalid SOV label for {base_name}: {line}")
                    return None
                vals = list(map(float, line.split()))
                sov_bbox = vals[1:]  # skip class
        except (ValueError, FileNotFoundError) as e:
            # print(f"[DEBUG] Error loading SOV label for {base_name}: {e}")
            return None

        sov_bbox = torch.tensor(sov_bbox, dtype=torch.float32)

        # 3) Combine features and SOV bbox -> Expected shape: [32 + 4] = [36]
        combined_input = torch.cat([features, sov_bbox], dim=0)
        # print(f"[DEBUG] Combined input shape for {base_name}: {combined_input.shape}")
        if combined_input.shape[0] != self.expected_feat_dim:
            # print(f"[DEBUG] Combined input shape {combined_input.shape[0]} does not match expected {self.expected_feat_dim} for {base_name}")
            return None

        # 4) Load Aorta bounding box (assumes format: class x y w h)
        aorta_txt = os.path.join(self.aorta_label_dir, base_name + ".txt")
        try:
            with open(aorta_txt, "r") as f:
                line = f.readline().strip()
                vals = list(map(float, line.split()))
                aorta_bbox = vals[1:]  # skip class
        except (ValueError, FileNotFoundError) as e:
            # print(f"[DEBUG] Error loading Aorta label for {base_name}: {e}")
            return None

        aorta_bbox = torch.tensor(aorta_bbox, dtype=torch.float32)

        return combined_input, aorta_bbox

# Update skip_invalid_collate_fn to filter out invalid samples
def skip_invalid_collate_fn(batch):
    valid_batch = [x for x in batch if x is not None]
    if len(valid_batch) == 0:
        # print("[DEBUG] skip_invalid_collate_fn: Batch is empty after filtering invalid samples.")
        return None
    from torch.utils.data.dataloader import default_collate
    return default_collate(valid_batch)


##############################################
# Create the BBFeaturesDataset and split into train/test
##############################################
# Update these paths as needed:
feature_dir = "extracted_features_total"  # or "extracted_features_train"
sov_label_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelSOV"
aorta_label_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelAscendingAorta"

# expected_feat_dim: 32 from pooled features + 4 for the SOV bbox = 36
expected_feat_dim = 512 + 4

full_bb_dataset = BBFeaturesDataset(
    feature_dir=feature_dir,
    sov_label_dir=sov_label_dir,
    aorta_label_dir=aorta_label_dir,
    expected_feat_dim=expected_feat_dim
)

print(f"[DEBUG] Total samples in full_bb_dataset: {len(full_bb_dataset)}")

# Split into train/test
train_size = int(0.8 * len(full_bb_dataset))
test_size = len(full_bb_dataset) - train_size
train_dataset, test_dataset = random_split(full_bb_dataset, [train_size, test_size],
                                          generator=torch.Generator().manual_seed(42))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=skip_invalid_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=skip_invalid_collate_fn)


[DEBUG] Total samples in full_bb_dataset: 1209


In [16]:
print(train_size)

967


In [17]:
# for i in range(5):
#     sample = full_bb_dataset[i]
#     print(i, sample)


In [18]:
# Debug: iterate over one batch from the train loader and print shapes
print("[DEBUG] Checking one batch from train_loader:")
count_train=0
count_test=0
for batch in train_loader:
    if batch is None:
        continue
        # print("[DEBUG] Received empty batch from train_loader.")
    else:
        count_train+=1
        inputs, targets = batch
        # print(f"[DEBUG] Train batch inputs shape: {inputs.shape}")
        # print(f"[DEBUG] Train batch targets shape: {targets.shape}")

# Debug: iterate over one batch from the test loader and print shapes
print("[DEBUG] Checking one batch from test_loader:")
for batch in test_loader:
    if batch is None:
        continue
        # print("[DEBUG] Received empty batch from test_loader.")
    else:
        count_test+=1
        inputs, targets = batch
        # print(f"[DEBUG] Test batch inputs shape: {inputs.shape}")
        # print(f"[DEBUG] Test batch targets shape: {targets.shape}")
print(count_train)
print(count_test)

[DEBUG] Checking one batch from train_loader:
[DEBUG] Checking one batch from test_loader:
964
241


In [19]:
print(test_size)

242


In [20]:
print(full_bb_dataset[0])

(tensor([ 7.1584e-02,  2.6673e-01,  3.5296e-01,  3.4407e-01,  8.4384e-02,  2.3904e-01,  2.0641e-01,  2.9894e-01,  6.6111e-01,  5.8709e-01,  1.5757e-01,  1.0350e+00,  2.7312e+00,  8.3184e-01,  2.4677e-01,  1.6019e+00,  2.8077e-01,  6.7877e-01,  5.7386e-01,  7.3079e-01,  4.1862e-01,  6.2889e-01,  4.9524e-01,  6.8352e-01,
         5.7770e-01,  5.1273e-01,  3.9516e-01,  9.5255e-01,  2.7095e+00,  1.0394e+00,  5.0559e-01,  1.6623e+00,  1.9263e+00,  2.2489e+00,  2.3104e+00,  2.2609e+00,  2.1886e+00,  2.5274e+00,  2.5764e+00,  2.4813e+00,  2.4544e+00,  2.5613e+00,  2.5888e+00,  2.4636e+00,  3.4011e+00,  2.7121e+00,  2.2931e+00,  2.0916e+00,
         5.7011e-01,  3.6994e-01,  3.1260e-01,  4.7662e-01,  6.6651e-01,  3.9207e-01,  3.4196e-01,  4.8202e-01,  6.5930e-01,  2.7941e-01,  2.9743e-01,  4.8515e-01,  3.5693e+00,  1.0667e+00,  2.6723e-01,  1.4673e+00,  2.5234e-02,  2.1282e-01,  3.5342e-01,  3.2693e-01,  1.2221e-01,  3.2764e-01,  3.7302e-01,  3.7246e-01,
         2.1866e-01,  2.3942e-01,  2.32

In [ ]:

##############################################
# Determine the input dimension dynamically from one sample
##############################################
sample, _ = full_bb_dataset[0]
in_dim = sample.shape[0]  # This should be 32 (features) + len(sov_bbox) (typically 4)
print("Determined input dimension:", in_dim)

##############################################
# 8) MLP Model Definition
##############################################
class AscAortaMLP(nn.Module):
    def __init__(self, in_dim=516, hidden1=256, hidden2=16, final_hidden1=128, final_hidden2=64, out_dim=4):
        super().__init__()
        
        # Layers for the first 512 values
        self.fc1 = nn.Linear(512, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)  # Produces a 16-dimensional vector
        self.relu = nn.ReLU()
        
        # Layers for the combined (16 + 4 = 20) vector
        self.fc3 = nn.Linear(hidden2 + 4, final_hidden1)
        self.fc4 = nn.Linear(final_hidden1, final_hidden2)
        self.fc5 = nn.Linear(final_hidden2, out_dim)

    def forward(self, x):
        """
        x: Input tensor of shape (batch_size, 516)
        """
        # Split x into two parts: the first 512 values and the last 4 values
        x1 = x[:, :512]  # Shape (batch_size, 512)
        x2 = x[:, 512:]  # Shape (batch_size, 4)
        
        # Pass x1 (512) through two layers
        x1 = self.relu(self.fc1(x1))
        x1 = self.relu(self.fc2(x1))  # Output is now 16-dimensional
        
        # Concatenate x1 (16) with x2 (4) to form a 20-dimensional vector
        combined = torch.cat([x1, x2], dim=1)
        
        # Pass the combined vector through the final layers
        x = self.relu(self.fc3(combined))
        x = self.relu(self.fc4(x))
        return self.relu(self.fc5(x))


model = AscAortaMLP(in_dim=in_dim, hidden1=512, hidden2=256, out_dim=4)
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

##############################################
# 9) Training Loop
##############################################

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        if batch is None:
            continue
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        preds = model(inputs)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader) if len(train_loader) > 0 else 0
    print(f"Epoch {epoch+1}/{epochs}, Loss = {avg_loss:.4f}")

##############################################

Determined input dimension: 516
Epoch 1/10, Loss = 0.0071
Epoch 2/10, Loss = 0.0051
Epoch 3/10, Loss = 0.0041
Epoch 4/10, Loss = 0.0029
Epoch 5/10, Loss = 0.0022
Epoch 6/10, Loss = 0.0013
Epoch 7/10, Loss = 0.0011
Epoch 8/10, Loss = 0.0010
Epoch 9/10, Loss = 0.0010
Epoch 10/10, Loss = 0.0010


In [22]:

##############################################
# 10) Evaluation (IoU + mAP@0.5)
##############################################
def box_iou(b1, b2, eps=1e-9):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    b1_xmin = x1 - w1/2
    b1_xmax = x1 + w1/2
    b1_ymin = y1 - h1/2
    b1_ymax = y1 + h1/2
    b2_xmin = x2 - w2/2
    b2_xmax = x2 + w2/2
    b2_ymin = y2 - h2/2
    b2_ymax = y2 + h2/2
    inter_xmin = max(b1_xmin, b2_xmin)
    inter_ymin = max(b1_ymin, b2_ymin)
    inter_xmax = min(b1_xmax, b2_xmax)
    inter_ymax = min(b1_ymax, b2_ymax)
    inter_w = max(0.0, inter_xmax - inter_xmin)
    inter_h = max(0.0, inter_ymax - inter_ymin)
    inter_area = inter_w * inter_h
    area1 = (b1_xmax - b1_xmin) * (b1_ymax - b1_ymin)
    area2 = (b2_xmax - b2_xmin) * (b2_ymax - b2_ymin)
    union = area1 + area2 - inter_area + eps
    return inter_area / union

def evaluate_bboxes(model, dataloader, iou_threshold=0.5, device="cuda"):
    model.eval()
    iou_list = []
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            if batch is None:
                continue
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            preds = model(inputs)
            preds = preds.cpu().numpy()
            targs = targets.cpu().numpy()
            for p, t in zip(preds, targs):
                iou_val = box_iou(p, t)
                iou_list.append(iou_val)
                if iou_val >= iou_threshold:
                    correct += 1
                total += 1
    mean_iou = float(np.mean(iou_list)) if len(iou_list) > 0 else 0.0
    accuracy = correct / total if total > 0 else 0.0
    map_50 = accuracy
    return mean_iou, accuracy, map_50

mean_iou, accuracy, map_50 = evaluate_bboxes(model, test_loader, iou_threshold=0.5, device=device)
print(f"Mean IoU: {mean_iou:.3f}")
print(f"Accuracy (IoU>=0.5): {accuracy:.3f}")
print(f"mAP@0.5 (single-box approximation): {map_50:.3f}")

Mean IoU: 0.631
Accuracy (IoU>=0.5): 0.859
mAP@0.5 (single-box approximation): 0.859


In [23]:

##############################################
# 11) Visualization of Ascending Aorta Bounding Boxes on Test Samples
##############################################
def draw_bbox_cv2(image, bbox, color, thickness=2, normalized=True):
    """
    Draws a rectangle on an OpenCV image given a YOLO-format bbox [x_center, y_center, w, h].
    If normalized=True, scales the coordinates by the image dimensions.
    """
    h, w = image.shape[:2]
    cx, cy, bw, bh = bbox
    if normalized:
        cx *= w
        cy *= h
        bw *= w
        bh *= h
    x1 = int(cx - bw/2)
    y1 = int(cy - bh/2)
    x2 = int(cx + bw/2)
    y2 = int(cy + bh/2)
    cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

def visualize_predictions_in_folder(model, backbone, neck,
                                     image_folder,
                                     sov_label_folder,
                                     aorta_label_folder=None,
                                     transform=None,
                                     device="cuda",
                                     save_folder="visualizations",
                                     show_result=False):
    if save_folder is not None:
        os.makedirs(save_folder, exist_ok=True)
    image_files = [f for f in os.listdir(image_folder)
                   if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    for img_name in image_files:
        img_path = os.path.join(image_folder, img_name)
        pil_img = Image.open(img_path).convert("RGB")
        if transform is None:
            transform = T.Compose([T.Resize((640,640)), T.ToTensor()])
        img_tensor = transform(pil_img).unsqueeze(0).to(device)
        
        # Extract features from the image
        backbone_outputs = backbone(img_tensor)
        neck_outputs = neck(backbone_outputs)
        if isinstance(neck_outputs, list):
            feats = neck_outputs[0]
        else:
            feats = neck_outputs
        # Global average pooling to get [1,32]
        feats = F.adaptive_avg_pool2d(feats, (4,4))
        feats = feats.view(feats.size(0), -1).squeeze(0)  # shape [32]
        
        # Load SOV label for this image
        base_name, _ = os.path.splitext(img_name)
        sov_txt = os.path.join(sov_label_folder, base_name + ".txt")
        if not os.path.exists(sov_txt):
            print(f"SOV label not found for {img_name}")
            continue
        with open(sov_txt, "r") as f:
            line = f.readline().strip()
            if not line.strip():  # If the line is empty, skip it
                print(f"Skipping empty line in {sov_txt}")
                continue
            if not all(part.replace('.', '', 1).isdigit() for part in line.split()):
                print(f"Skipping invalid line in {sov_txt}: {line.strip()}")
                continue
            vals = list(map(float, line.split()))
            sov_bbox = vals[1:]  # skip class


        sov_bbox = torch.tensor(sov_bbox, dtype=torch.float32, device=device)
        
        # Create combined input: features [32] + sov_bbox [4] = [36]
        combined_input = torch.cat([feats, sov_bbox], dim=0).unsqueeze(0)
        model.eval()
        with torch.no_grad():
            pred_bbox = model(combined_input)
        pred_bbox = pred_bbox.squeeze(0).cpu().numpy()  # [4]
        
        # Optionally load ground truth aorta bounding box if available
        gt_bbox = None
        if aorta_label_folder is not None:
            aorta_txt = os.path.join(aorta_label_folder, base_name + ".txt")
            if os.path.exists(aorta_txt):
                with open(aorta_txt, "r") as f:
                    line = f.readline().strip()
                    vals = list(map(float, line.split()))
                    gt_bbox = np.array(vals[1:])  # skip class
        
        # Convert PIL image to OpenCV image (BGR)
        image_cv = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        # Draw predicted bounding box in red
        draw_bbox_cv2(image_cv, pred_bbox, color=(0,0,255), thickness=2, normalized=True)
        # Draw ground truth bounding box in green if available
        # if gt_bbox is not None:
        #     draw_bbox_cv2(image_cv, gt_bbox, color=(0,255,0), thickness=2, normalized=True)
        
        if save_folder is not None:
            out_path = os.path.join(save_folder, base_name + "_vis.png")
            cv2.imwrite(out_path, image_cv)
            print(f"Saved visualization to {out_path}")
        if show_result:
            cv2.imshow("Prediction", image_cv)
            cv2.waitKey(0)
    if show_result:
        cv2.destroyAllWindows()
model=
# Visualize predictions on test samples
visualize_predictions_in_folder(
    model=model,
    backbone=backbone,
    neck=neck,
    image_folder=test_image_folder,
    sov_label_folder="/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelSOV",
    aorta_label_folder="/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelAscendingAorta",
    transform=T.Compose([T.Resize((640,640)), T.ToTensor()]),
    device=device,
    save_folder="acending_aorta_output_visual",
    show_result=False
)




Saved visualization to acending_aorta_output_visual/P G - 3mensio Screen Recording_longitudinal_view_242_brightness_contrast_vis.png
Saved visualization to acending_aorta_output_visual/P G - 3mensio Screen Recording_longitudinal_view_242_clahe_vis.png
Saved visualization to acending_aorta_output_visual/P G - 3mensio Screen Recording_longitudinal_view_245_vis.png
Saved visualization to acending_aorta_output_visual/P G - 3mensio Screen Recording_longitudinal_view_245_brightness_contrast_vis.png
Saved visualization to acending_aorta_output_visual/P G - 3mensio Screen Recording_longitudinal_view_242_scaling_vis.png
Saved visualization to acending_aorta_output_visual/P G - 3mensio Screen Recording_longitudinal_view_245_blur_vis.png
Saved visualization to acending_aorta_output_visual/P G - 3mensio Screen Recording_longitudinal_view_245_clahe_vis.png
Saved visualization to acending_aorta_output_visual/P G - 3mensio Screen Recording_longitudinal_view_245_scaling_vis.png
Saved visualization to 

In [24]:
def save_predicted_bboxes_to_txt(model, backbone, neck, image_folder, sov_label_folder, output_folder, transform=None, device="cuda"):
    """
    Predict ascending aorta bounding boxes and save them as .txt files in the specified output folder.
    
    Args:
        model (nn.Module): The trained model to predict bounding boxes.
        backbone (nn.Module): Backbone of YOLOv8.
        neck (nn.Module): Neck of YOLOv8.
        image_folder (str): Path to the folder containing input images.
        sov_label_folder (str): Path to the folder containing SOV labels.
        output_folder (str): Path to save predicted bounding boxes in .txt format.
        transform (callable, optional): Transformations applied to input images.
        device (str): Device to run the model on.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)
    
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
    for img_name in image_files:
        img_path = os.path.join(image_folder, img_name)
        pil_img = Image.open(img_path).convert("RGB")
        if transform is None:
            transform = T.Compose([T.Resize((640, 640)), T.ToTensor()])
        img_tensor = transform(pil_img).unsqueeze(0).to(device)

        # Extract features from the image
        backbone_outputs = backbone(img_tensor)
        neck_outputs = neck(backbone_outputs)
        feats = neck_outputs[0] if isinstance(neck_outputs, list) else neck_outputs
        feats = F.adaptive_avg_pool2d(feats, (4, 4)).view(feats.size(0), -1).squeeze(0)

        # Load SOV label for this image
        base_name, _ = os.path.splitext(img_name)
        sov_txt = os.path.join(sov_label_folder, base_name + ".txt")
        if not os.path.exists(sov_txt):
            print(f"SOV label not found for {img_name}")
            continue
        
        with open(sov_txt, "r") as f:
            line = f.readline().strip()
            if not line.strip():  # If the line is empty, skip it
                print(f"Skipping empty line in {sov_txt}")
                continue
            
            # Check if all parts of the line can be converted to float
            if not all(part.replace('.', '', 1).isdigit() for part in line.split()):
                print(f"Skipping invalid line in {sov_txt}: {line.strip()}")
                continue
            
            # If the line passes the checks, parse it into floats
            vals = list(map(float, line.split()))
            sov_bbox = vals[1:]  # skip class
            sov_bbox = torch.tensor(sov_bbox, dtype=torch.float32, device=device)
        
        sov_bbox = torch.tensor(sov_bbox, dtype=torch.float32, device=device)

        # Combine features and SOV bounding box
        combined_input = torch.cat([feats, sov_bbox], dim=0).unsqueeze(0)

        # Predict ascending aorta bounding box
        model.eval()
        with torch.no_grad():
            pred_bbox = model(combined_input).squeeze(0).cpu().numpy()  # [4]

        # Save the predicted bounding box to a .txt file
        output_txt_path = os.path.join(output_folder, base_name + ".txt")
        with open(output_txt_path, "w") as txt_file:
            txt_file.write(f"0 {pred_bbox[0]:.6f} {pred_bbox[1]:.6f} {pred_bbox[2]:.6f} {pred_bbox[3]:.6f}\n")
        
        print(f"Saved predicted bbox to {output_txt_path}")
    
# Visualize predictions on test samples
save_predicted_bboxes_to_txt(
    model=model,
    backbone=backbone,
    neck=neck,
    image_folder="/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/totalImages",
    sov_label_folder="/mnt/nvme_disk2/User_data/hp927k/phase2/AscendingAortaBBprediction/labelSOV",
    output_folder="AOTBB",
    
)


/tmp/ipykernel_2048424/1401593929.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sov_bbox = torch.tensor(sov_bbox, dtype=torch.float32, device=device)


Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_471.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_471_blur.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_471_brightness_contrast.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_471_clahe.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_471_scaling.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_474.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_474_blur.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_474_brightness_contrast.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_474_clahe.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen Recording_longitudinal_view_474_scaling.txt
Saved predicted bbox to AOTBB/A A - 3mensio Screen R

In [26]:
torch.save(model.state_dict(),"ascending_aorta_model_without_manual_loss.pt")

In [ ]:
###